In [ ]:
import pandas as pd
import numpy as np
import math
import pickle
import collections

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path


import matplotlib.pyplot as plt

import SMOTE
import feature_selector
import DE
import CFS
import birch
import metrics.abcd
import birch_bellwether

import metrices
import measures

import sys
import traceback
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def cluster_driver(df,print_tree = False):
    X = df.apply(pd.to_numeric)
    cluster = birch.birch(branching_factor=20)
        #X.set_index('Project Name',inplace=True)
    cluster.fit(X)
    cluster_tree,max_depth = cluster.get_cluster_tree()
        #cluster_tree = cluster.model_adder(cluster_tree)
    if print_tree:
        cluster.show_clutser_tree()
    return cluster,cluster_tree,max_depth

def build_BIRCH(attr_df):
    cluster,cluster_tree,_ = cluster_driver(attr_df)
    return cluster,cluster_tree

In [ ]:
#get cluster wise data for summarzation using median
def find_bellwether(data_source1,other_projects,path,fold):
    goals = ['f1','g','recall','precision','pf']
    for goal in goals:
        df = pd.read_csv(data_source1 + '/bellwether_' + goal + '_0.csv')
        _bellwether_projects = df.bellwether.values.tolist()
        bell = birch_bellwether.bellwether(path,df)
        final_score = bell.bellwether(_bellwether_projects,other_projects)
        #print(_bellwether_projects,other_projects)
        _results = {}
        for s_project in final_score.keys():
            if s_project not in _results.keys():
                _results[s_project] = {}
            for d_projects in final_score[s_project].keys():
                if goal == 'g':
                    _goal = 'g-score'
                else:
                    _goal = goal
                _results[s_project][d_projects] = np.median(final_score[s_project][d_projects][_goal])
        _df_results = pd.DataFrame.from_dict(_results, orient='index')
        _df_results = _df_results.to_csv('_temp_results'  + str(fold) + '.csv')
        _df_results = pd.read_csv('_temp_results'  + str(fold) + '.csv')
        med_count = []
        counts = []
        score_med = []
        for row in range(_df_results.shape[0]):
            j = _df_results.iloc[row].values[1:]
            j_med = np.median(j)
            if goal != 'pf':
                value = sum(i >= 0.66 for i in j)
            else:
                value = sum(i <= 0.33 for i in j)
            med_count.append(j_med)
            counts.append(value)
        print(_df_results)
        if goal == 'pf':
            score_med.append([counts[med_count.index(min(med_count))],round(np.mean(counts),2),
                                      max(med_count),_df_results.iloc[med_count.index(min(med_count)),0]])     
        elif goal == 'g':
            score_med.append([counts[med_count.index(max(med_count))],round(np.mean(counts),2),
                                      max(med_count),_df_results.iloc[med_count.index(max(med_count)),0]])
        else:
            score_med.append([counts[med_count.index(max(med_count))],round(np.mean(counts),2),
                                      max(med_count),_df_results.iloc[med_count.index(max(med_count)),0]])
        score_df = pd.DataFrame(score_med, columns = ['max_bell','avge','median','bellwether'])
        score_df.to_csv(data_source1 + '/results/bellwether_' + goal + '_0_perf.csv')
        

In [ ]:
for i in range(1,10):
    print(i)
    path = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/data/1385/converted'
    data_source1 = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/exp2/2/fold_' + str(i)
    if platform.system() == 'Darwin' or platform.system() == 'Linux':
        _dir = path + '/'
    else:
        _dir = path + '\\'

    projects = [f for f in listdir(_dir) if isfile(join(_dir, f))]
    #print(projects)
    find_bellwether(data_source1,projects,path,i)
    